In [1]:
import yfinance as yf
import pandas as pd
import datetime
from fredapi import Fred
import numpy as np
import pandas_ta as ta
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import load_model
import pickle

pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",100)

2024-12-12 14:59:00.876638: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
start_date = (datetime.datetime.today() - datetime.timedelta(days=100)).strftime('%Y-%m-%d')
end_date = datetime.datetime.today().strftime('%Y-%m-%d')

print(start_date,end_date, type(end_date),type(end_date))
# Example list of S&P 500 tickers (full list can be obtained elsewhere)
sp500_tickers = ["AAPL", "NVDA", "MSFT", "GOOG", "GOOGL", "AMZN", "META", "AVGO", "LLY", "TSLA", 
                "WMT", "JPM", "V", "XOM", "UNH", "ORCL", "MA", "HD", "PG", "COST", "JNJ", 
                "NFLX", "ABBV", "BAC", "KO", "CRM", "CVX", "MRK", "TMUS", "AMD", "PEP", 
                "ACN", "LIN", "TMO", "MCD", "CSCO", "ADBE", "WFC", "IBM", "GE", "ABT", 
                "DHR", "AXP", "MS", "CAT", "NOW", "QCOM", "PM", "ISRG", "VZ"]

# Download data for all tickers at once
stock_price_data = yf.download(sp500_tickers, start=start_date, end=end_date, group_by='ticker')

2024-09-03 2024-12-12 <class 'str'> <class 'str'>


[*********************100%***********************]  50 of 50 completed


In [3]:
stock_data = stock_price_data.copy()

In [4]:
# Flatten MultiIndex columns in stock_data
stock_data.columns = ['_'.join(col).strip() for col in stock_data.columns.values]

In [5]:
stock_data.tail()

,ABBV_Open,ABBV_High,ABBV_Low,ABBV_Close,ABBV_Adj Close,ABBV_Volume,NOW_Open,NOW_High,NOW_Low,NOW_Close,NOW_Adj Close,NOW_Volume,NFLX_Open,NFLX_High,NFLX_Low,NFLX_Close,NFLX_Adj Close,NFLX_Volume,TMO_Open,TMO_High,TMO_Low,TMO_Close,TMO_Adj Close,TMO_Volume,AMZN_Open,AMZN_High,AMZN_Low,AMZN_Close,AMZN_Adj Close,AMZN_Volume,META_Open,META_High,META_Low,META_Close,META_Adj Close,META_Volume,AMD_Open,AMD_High,AMD_Low,AMD_Close,AMD_Adj Close,AMD_Volume,MSFT_Open,MSFT_High,MSFT_Low,MSFT_Close,MSFT_Adj Close,MSFT_Volume,WMT_Open,WMT_High,WMT_Low,WMT_Close,WMT_Adj Close,WMT_Volume,BAC_Open,BAC_High,BAC_Low,BAC_Close,BAC_Adj Close,BAC_Volume,V_Open,V_High,V_Low,V_Close,V_Adj Close,V_Volume,LIN_Open,LIN_High,LIN_Low,LIN_Close,LIN_Adj Close,LIN_Volume,ADBE_Open,ADBE_High,ADBE_Low,ADBE_Close,ADBE_Adj Close,ADBE_Volume,UNH_Open,UNH_High,UNH_Low,UNH_Close,UNH_Adj Close,UNH_Volume,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Adj Close,AAPL_Volume,PG_Open,PG_High,PG_Low,PG_Close,PG_Adj Close,PG_Volume,GE_Open,GE_High,GE_Low,GE_Close,GE_Adj Close,GE_Volume,ABT_Open,ABT_High,ABT_Low,ABT_Close,ABT_Adj Close,ABT_Volume,GOOG_Open,GOOG_High,GOOG_Low,GOOG_Close,GOOG_Adj Close,GOOG_Volume,KO_Open,KO_High,KO_Low,KO_Close,KO_Adj Close,KO_Volume,LLY_Open,LLY_High,LLY_Low,LLY_Close,LLY_Adj Close,LLY_Volume,HD_Open,HD_High,HD_Low,HD_Close,HD_Adj Close,HD_Volume,PEP_Open,PEP_High,PEP_Low,PEP_Close,PEP_Adj Close,PEP_Volume,MRK_Open,MRK_High,MRK_Low,MRK_Close,MRK_Adj Close,MRK_Volume,VZ_Open,VZ_High,VZ_Low,VZ_Close,VZ_Adj Close,VZ_Volume,XOM_Open,XOM_High,XOM_Low,XOM_Close,XOM_Adj Close,XOM_Volume,IBM_Open,IBM_High,IBM_Low,IBM_Close,IBM_Adj Close,IBM_Volume,MS_Open,MS_High,MS_Low,MS_Close,MS_Adj Close,MS_Volume,GOOGL_Open,GOOGL_High,GOOGL_Low,GOOGL_Close,GOOGL_Adj Close,GOOGL_Volume,CVX_Open,CVX_High,CVX_Low,CVX_Close,CVX_Adj Close,CVX_Volume,AVGO_Open,AVGO_High,AVGO_Low,AVGO_Close,AVGO_Adj Close,AVGO_Volume,CAT_Open,CAT_High,CAT_Low,CAT_Close,CAT_Adj Close,CAT_Volume,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Adj Close,TSLA_Volume,ISRG_Open,ISRG_High,ISRG_Low,ISRG_Close,ISRG_Adj Close,ISRG_Volume,ACN_Open,ACN_High,ACN_Low,ACN_Close,ACN_Adj Close,ACN_Volume,QCOM_Open,QCOM_High,QCOM_Low,QCOM_Close,QCOM_Adj Close,QCOM_Volume,NVDA_Open,NVDA_High,NVDA_Low,NVDA_Close,NVDA_Adj Close,NVDA_Volume,MA_Open,MA_High,MA_Low,MA_Close,MA_Adj Close,MA_Volume,PM_Open,PM_High,PM_Low,PM_Close,PM_Adj Close,PM_Volume,COST_Open,COST_High,COST_Low,COST_Close,COST_Adj Close,COST_Volume,AXP_Open,AXP_High,AXP_Low,AXP_Close,AXP_Adj Close,AXP_Volume,TMUS_Open,TMUS_High,TMUS_Low,TMUS_Close,TMUS_Adj Close,TMUS_Volume,CRM_Open,CRM_High,CRM_Low,CRM_Close,CRM_Adj Close,CRM_Volume,JPM_Open,JPM_High,JPM_Low,JPM_Close,JPM_Adj Close,JPM_Volume,ORCL_Open,ORCL_High,ORCL_Low,ORCL_Close,ORCL_Adj Close,ORCL_Volume,DHR_Open,DHR_High,DHR_Low,DHR_Close,DHR_Adj Close,DHR_Volume,CSCO_Open,CSCO_High,CSCO_Low,CSCO_Close,CSCO_Adj Close,CSCO_Volume,MCD_Open,MCD_High,MCD_Low,MCD_Close,MCD_Adj Close,MCD_Volume,JNJ_Open,JNJ_High,JNJ_Low,JNJ_Close,JNJ_Adj Close,JNJ_Volume,WFC_Open,WFC_High,WFC_Low,WFC_Close,WFC_Adj Close,WFC_Volume
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-12-05,174.979996,176.720001,173.529999,176.169998,176.169998,5444200,1115.099976,1130.599976,1113.109985,1123.719971,1123.719971,1075500,917.849976,927.000000,911.109985,917.869995,917.869995,2835300,527.210022,527.409973,517.679993,520.859985,520.859985,2665300,218.029999,222.149994,217.300003,220.550003,220.550003,41140200,617.080017,619.900024,607.049988,608.929993,608.929993,8081200,143.679993,143.949997,140.380005,141.360001,141.360001,29576900,437.920013,444.660004,436.170013,442.619995,442.619995,21697800,94.889999,95.599998,94.339996,95.300003,95.300003,21932300,46.56000

In [6]:
# Convert index to DatetimeIndex if not already
if not isinstance(stock_data.index, pd.DatetimeIndex):
    stock_data.index = pd.to_datetime(stock_data.index)

# Sort by date
stock_data.sort_index(inplace=True)

In [7]:
print(f"Stock data date range: {stock_data.index.min()} to {stock_data.index.max()}")

Stock data date range: 2024-09-03 00:00:00 to 2024-12-11 00:00:00


In [8]:
tickers = ["AAPL", "NVDA", "MSFT", "GOOG", "GOOGL", "AMZN", "META", "AVGO", "LLY", "TSLA",
                 "WMT", "JPM", "V", "XOM", "UNH", "ORCL", "MA", "HD", "PG", "COST", "JNJ",
                 "NFLX", "ABBV", "BAC", "KO", "CRM", "CVX", "MRK", "TMUS", "AMD", "PEP",
                 "ACN", "LIN", "TMO", "MCD", "CSCO", "ADBE", "WFC", "IBM", "GE", "ABT",
                 "DHR", "AXP", "MS", "CAT", "NOW", "QCOM", "PM", "ISRG", "VZ"]

In [9]:
with open('updated_all_dfs.pkl', 'rb') as file:
    loaded_data = pickle.load(file)

In [10]:
inference_data = {}

for comp in tickers:
    ll = 'df_'+comp
    reg = '^'+comp+'_'
    filtered_data = stock_data.filter(regex=reg)
    last_rows = loaded_data[ll].tail(filtered_data.shape[0])
    ind_ = filtered_data.index
    last_rows.index = ind_
    merged_df = filtered_data.join(last_rows, how='left', lsuffix='', rsuffix='_r')
    merged_df = merged_df[[col for col in merged_df.columns if not col.endswith('_r')]]
    inference_data[ll] = merged_df

In [11]:
new_model = load_model('lstm_model_general.h5')

2024-12-12 14:59:03.582137: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024-12-12 14:59:03.715030: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-12-12 14:59:03.715711: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-12-12 14:59:03.7

In [12]:
def prepare_inference_data(company_df, sequence_length=60):
    """
    Prepare input data for inference for a single company.
    Args:
        company_df (DataFrame): The DataFrame for a specific company.
        sequence_length (int): The number of past days to consider as input.

    Returns:
        numpy array: The input data ready for prediction.
    """
    # Ensure data is sorted by date
    company_df = company_df.sort_index()

    # Select relevant input features (exclude targets)
    input_features = company_df.filter(regex="^(?!.*target).*").values

    # Take the last `sequence_length` days as input for prediction
    if len(input_features) >= sequence_length:
        input_sequence = input_features[-sequence_length:]
        return np.expand_dims(input_sequence, axis=0)  # Add batch dimension
    else:
        raise ValueError("Insufficient data for inference (less than sequence length).")


In [13]:
def get_predictions_for_all_companies(all_dfs, model, sequence_length=60):
    """
    Get predictions for all companies using the trained model.
    Args:
        all_dfs (dict): Dictionary of company DataFrames.
        model: Trained LSTM model.
        sequence_length (int): Number of past days to consider as input.

    Returns:
        dict: Predictions for each company.
    """
    predictions = {}
    
    for company, df in all_dfs.items():
        try:
            # Prepare data for inference
            input_data = prepare_inference_data(df, sequence_length=sequence_length)
            
            # Make predictions
            pred = model.predict(input_data)
            
            # Store predictions
            predictions[company] = pred.flatten()  # Flatten the array for readability
        
        except ValueError as e:
            print(f"Skipping {company}: {e}")

    return predictions


In [14]:
# Get predictions for all companies
predictions = get_predictions_for_all_companies(inference_data, new_model, sequence_length=60)

2024-12-12 14:59:04.872978: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-12-12 14:59:04.873939: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-12-12 14:59:04.874668: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 35ms/step


In [15]:
type(predictions)

dict

In [16]:
len(list(predictions.keys()))

50

In [17]:
predictions.keys()

dict_keys(['df_AAPL', 'df_NVDA', 'df_MSFT', 'df_GOOG', 'df_GOOGL', 'df_AMZN', 'df_META', 'df_AVGO', 'df_LLY', 'df_TSLA', 'df_WMT', 'df_JPM', 'df_V', 'df_XOM', 'df_UNH', 'df_ORCL', 'df_MA', 'df_HD', 'df_PG', 'df_COST', 'df_JNJ', 'df_NFLX', 'df_ABBV', 'df_BAC', 'df_KO', 'df_CRM', 'df_CVX', 'df_MRK', 'df_TMUS', 'df_AMD', 'df_PEP', 'df_ACN', 'df_LIN', 'df_TMO', 'df_MCD', 'df_CSCO', 'df_ADBE', 'df_WFC', 'df_IBM', 'df_GE', 'df_ABT', 'df_DHR', 'df_AXP', 'df_MS', 'df_CAT', 'df_NOW', 'df_QCOM', 'df_PM', 'df_ISRG', 'df_VZ'])